### Starting AWS instance and running cifar10

In [40]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [41]:
from aws_setup import *

#### Define parameters

In [15]:
vpc_name='fast-ai'

#### Get Existing VPC by tag name

In [16]:
vpc = get_vpc(vpc_name); vpc

ec2.Vpc(id='vpc-6e6b2a17')

#### Request Spot instance

In [23]:
instance_name = f'{vpc_name}-instance'
# instance_type = 't2.micro'
instance_type = 't2.2xlarge'
# instance_type = 'g2.2xlarge'

In [24]:
spot_price = get_spot_prices()[instance_type]
bid_price = "%.4f" % (float(spot_price)*3)
print(f'Spot price: {spot_price}, Bid price: {bid_price}')

Spot price: 0.173400, Bid price: 0.5202


In [25]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type).build()

In [26]:
launch_specs['BlockDeviceMappings'][0]['Ebs']['VolumeSize'] = 80

In [27]:
launch_specs

{'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
   'Ebs': {'DeleteOnTermination': True,
    'VolumeSize': 80,
    'VolumeType': 'gp2'}}],
 'ImageId': 'ami-8c4288f4',
 'InstanceType': 't2.2xlarge',
 'KeyName': 'aws-key-fast-ai',
 'NetworkInterfaces': [{'AssociatePublicIpAddress': True,
   'DeviceIndex': 0,
   'Groups': ['sg-f60fca88'],
   'SubnetId': 'subnet-f056ff89'}]}

In [28]:
instance = create_spot_instance(instance_name, launch_specs, spot_price=bid_price); instance

Waiting on spot fullfillment...
Spot instance request failed: Your spot request is fulfilled.
Cancelling request. Please try again or use on demand.


Request on demand instance (if spot error)

In [ ]:
if not instance:
    instance = create_instance(instance_name, vpc, instance_type=instance_type); instance

In [32]:
if not instance:
    instance = get_instance(instance_name); instance

ec2.Instance(id='i-022bf4d0d7e5f9585')

### SSH

In [44]:
client = connect_to_instance(instance)

Connecting to SSH...
Connected!


#### Mount EFS

In [65]:
efs_addr = get_efs_address('fast-ai-efs'); efs_addr

'fs-0ea233a7.efs.us-west-2.amazonaws.com'

In [66]:
run_command(client, 'mkdir ~/efs_mount_point')

run_command returned: 
mkdir: cannot create directory ‘/home/ubuntu/efs_mount_point’: File exists



In [67]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [68]:
run_command(client, efs_mount_cmd)

run_command returned: 
mount.nfs: /home/ubuntu/efs_mount_point is busy or already mounted



In [69]:
run_command(client, 'ls efs_mount_point') # no reformatting

run_command returned: 
efs_saved.txt



### Upload cifar scripts

In [52]:
out, _ = run_command(client, 'mkdir cifar10')

run_command returned: 



In [70]:
upload_path = Path.cwd()/'upload_scripts/run_cifar10.sh'
upload_file(client, str(upload_path), 'cifar10/run_cifar10.sh')

In [71]:
upload_path = Path.cwd()/'upload_scripts/cifar10.py'
upload_file(client, str(upload_path), 'cifar10/cifar10.py')

## TMUX

In [73]:
tsess = TmuxSession(client, 'sess')

In [74]:
tsess.run_cmd('cd cifar10')

In [75]:
tsess.run_cmd('bash run_cifar10.sh')

# Testing


In [77]:
instance.modify_attribute(InstanceInitiatedShutdownBehavior={'Value': 'terminate'})


ClientError: An error occurred (UnsupportedOperation) when calling the ModifyInstanceAttribute operation: Modifying 'instanceInitiatedShutdownBehavior' is not supported for spot instances.

In [78]:
instance

ec2.Instance(id='i-022bf4d0d7e5f9585')